# 어텐션

## encoder 개선
* seq2seq은 encoder의 입력 문장과 상관없이, 고정 길이 벡터를 decoder로 전달한다
* 입력 문장이 길수록 손실이 발생한다 -> 출력의 길이를 입력 문장의 길이에 따라 바꾸어야 한다
* 해결법: 각 시각의 은닉 상태 벡터를 모두 이용한다

## decoder 개선
* 목표: 도착어 단어와 대응하는 출발어 단어의 정보를 골라내기 -> 어텐션
  * 예: Decoder가 "I"를 출력할 때, hs에서 "나"에 대응하는 벡터를 선택해야 함
* LSTM과 Affine계층 사이 새로운 계층 추가
  * 입력은 2가지: Encoder로부터 받는 hs 행렬 + 시각별 LSTM 계층의 은닉상태
  * 각 단어의 가중치를 별도로 계산 후, hs의 행벡터에 대한 가중합 계산

In [3]:
import numpy as np
T, H = 5, 4 # 시계열의 길이, 은닉상태 원소 수
hs = np.random.randn(T, H)
a = np.array([.8, .1, .03, .05, .02])

ar = a.reshape(T, 1).repeat(H, axis=1)
print(ar.shape)

t = hs * ar # 원소별 곱을 계산한다.
print(t.shape)

c = np.sum(t, axis=0) # 가중합을 계산한다. (axis=0 -> 0번째 축이 사라진다.)
print(c.shape)

(5, 4)
(5, 4)
(4,)


In [4]:
# 미니배치 처리용
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T) # sequence별로 가중치는 다르겠지
ar = a.reshape(N, T, 1).repeat(H, axis=2)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1)
print(c.shape)

(10, 5, 4)
(10, 4)


가중합 a 구하는 방법
* LSTM의 은닉상태 벡터를 $\bf{h}$라 할 때, $\bf{h}$가 $\bf{hs}$의 단어벡터와 얼마나 비슷한지 파악하기 -> 내적
* h와 hs의 각 행벡터 간 내적을 구하고, 소프트맥스 함수로 정규화

In [6]:
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)

t = hs * hr
s = np.sum(t, axis=2)
softmax = Softmax()
a = softmax.forward(s)
print(a.shape)

(10, 5)


구현은 `attention_layer.py`를 보세요.

## Attention을 갖춘 seq2seq 구현

* Encoder: **모든 시간**의 은닉계층을 반환해야 한다는 점 외에는 Decoder와 동일하다
* Decoder: LSTM과 Affine layer 사이에 Attention Layer가 포함되며, Affine Layer는 LSTM의 은닉상태 벡터와 Attention Layer의 맥락벡터를 concatenate해서 입력한다.

`attention_seq2seq.py` 를 보세요.